In [1]:
from typing import Optional
import numpy as np
import numpy.typing as npt

In [2]:
def cgd(A, b, x0: Optional[npt.NDArray] = None):
    """Run conjugate gradient descent to solve x in Ax = b.
    """
    dim  = A.shape[0]
    if x0 is None:
        x = np.zeros(dim)
    else:
        x = x0
    r    = b - A @ x
    p    = r
    rr   = r @ r
    xs   = [x]
        
    for i in range(1, dim):

        # The only matrix-vector multiplication per iteration!
        Ap     = A @ p
        alpha  = rr / (p @ Ap)
        x      = x + alpha * p
        r      = r - alpha * Ap
        rr_new = r @ r
        beta   = rr_new / rr
        p      = r + beta * p
        rr     = rr_new

        xs.append(x)

    return np.array(xs)

In [3]:
def generate_positive_definite_matrix(n):
    """
    Generates a random positive definite symmetric matrix using eigen decomposition.
    """
    # Step 1: Generate a random orthogonal matrix via QR decomposition
    Q, _ = np.linalg.qr(np.random.randn(n, n))
    
    # Step 2: Create a diagonal matrix with positive entries (eigenvalues)
    eigenvalues = np.random.uniform(low=0.1, high=10.0, size=n)
    D = np.diag(eigenvalues)
    
    # Step 3: Construct the positive definite matrix
    pd_matrix = Q @ D @ Q.T
    
    return pd_matrix

### Generate example positive definite matrix `A` and vector `b`.

In [4]:
np.random.seed(0)
dim = 5
A = generate_positive_definite_matrix(dim)
# Eigenvalues of the matrix
eigenvalues = np.linalg.eigvals(A)

print("Eigenvalues:\n", eigenvalues)
print("Is the matrix positive definite?", np.all(eigenvalues > 0))

b = np.random.randn(dim)

Eigenvalues:
 [0.28601902 9.44310598 6.15974765 6.20764657 6.21459142]
Is the matrix positive definite? True


In [5]:
A_inv = np.linalg.inv(A)
x_true = A_inv @ b
print(x_true)
print(cgd(A, b)[-1])

[ 0.17015063 -0.35800967  0.27814836  0.32822873 -0.69968513]
[ 0.17015524 -0.35800289  0.27814979  0.32823596 -0.69968351]


In [19]:
def naive_cgd(A, b, x0: Optional[npt.NDArray] = None):
    dim = A.shape[0]
    xs = []
    rs = []
    if x0 is None:
        x = np.zeros(dim)
    else:
        x = x0
    for i in range(1, dim + 1):
        r = b - A @ x
        rs.append(r)
        if i == 1:
            p = r
            P = np.reshape(p, (dim, 1))
        else:
            AP = A @ P  # (n, n)
            rTAP = AP.T @ r  # (1, d)
            pTAp_all = np.multiply(P, AP).sum(axis=0)  # (d,)
            alphas = np.divide(rTAP, pTAp_all)
            print(alphas)
            p = r - P @ alphas  # (n,)
            P = np.concatenate((P, np.reshape(p, (dim, 1))), axis=1)
        x = x + np.dot(r, p) / np.dot(p, A @ p) * p
        xs.append(x)

    return np.array(xs), np.array(rs), P.T


In [20]:
xs, rs, P = naive_cgd(A, b)

[-0.03471906]
[-6.13068816e-17 -1.68992559e-01]
[-8.38329697e-17 -6.45662577e-16 -8.45218913e-02]
[ 3.00617568e-17 -1.42616890e-15  7.81032247e-16 -1.10411775e-06]


In [21]:
xs[-1]

array([ 0.17015063, -0.35800967,  0.27814836,  0.32822873, -0.69968513])

In [23]:
rs @ rs.T

array([[ 8.74465495e+00, -8.32667268e-16, -5.55111512e-16,
        -9.57567359e-16, -1.04605181e-16],
       [-8.32667268e-16,  3.03606201e-01, -6.93889390e-17,
        -2.32019265e-16, -3.67470209e-16],
       [-5.55111512e-16, -6.93889390e-17,  5.13071889e-02,
        -1.90819582e-17,  6.55283746e-17],
       [-9.57567359e-16, -2.32019265e-16, -1.90819582e-17,
         4.33658064e-03,  2.54366112e-17],
       [-1.04605181e-16, -3.67470209e-16,  6.55283746e-17,
         2.54366112e-17,  4.78809568e-09]])

In [24]:
P @ A @ P.T

array([[ 5.79496720e+01, -8.81875175e-16, -1.42530681e-16,
        -1.82705279e-16,  1.13203712e-19],
       [-1.47002111e-16,  2.26760333e+00,  8.33424257e-17,
        -8.78790840e-19, -2.10528647e-20],
       [ 1.30484168e-16,  5.39873547e-17,  1.74360525e-02,
        -9.25555829e-18,  7.31801390e-21],
       [-9.29450886e-17,  3.07234444e-17, -2.51495096e-18,
         2.66377675e-02,  2.90959839e-21],
       [-2.71050543e-20, -3.38813179e-21,  1.69406589e-21,
         1.69406589e-21,  2.97433574e-08]])